In [1]:
import sys
sys.path.append(r'..//')
from ioMicro import *

In [2]:
master_data_folder = r'\\192.168.0.21\bbfishdc21\HttHuman_Snonia__06_12_2023'
#master_data_folder = r'\\192.168.0.21\bbfishdc21\HttHuman_Snonia__06_12_2023'
#master_data_folder = r'\\192.168.0.96\bbfish124\HttHuman_Snonia__06_12_2023'

In [6]:
fl =r'\\192.168.0.21\bbfishdc21\HttHuman_Snonia__06_12_2023\H0__set5\Conv_zscan__30.zarr'
im = read_im(fl)
import napari
napari.view_image(im);

In [7]:
im.shape

(4, 37, 3000, 3000)

In [11]:
im_dapi = np.array(im[-1],dtype=np.float32)
im_bk = np.array(im[-2],dtype=np.float32)

In [24]:
ifr = len(im_dapi)//2
fr = im_bk[ifr]/im_dapi[ifr]
fr_ = np.max(fr)
v = napari.view_image(im_dapi[ifr]-im_bk[ifr]/fr_)
v.add_image(im_dapi[ifr])

<Image layer 'Image [1]' at 0x20f9b5f6310>

In [25]:
imd = im_dapi-im_bk/fr_

In [26]:
psf_file = r'C:\Scripts\NMERFISH\psfs\psf_750_Scope3_final.npy'
psf = np.load(psf_file)
imd_ = full_deconv(imd,s_=500,pad=100,psf=psf,parameters={'method': 'wiener', 'beta': 0.01},gpu=True,force=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:07<00:00,  4.69it/s]


In [29]:
napari.view_image(imd_[:,::3,::3])

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 499.5, 499.5), zoom=1.14095, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 37.0, 1.0), (0.0, 1000.0, 1.0), (0.0, 1000.0, 1.0)), current_step=(18, 500, 500), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x2105b439070>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10, unit=None), text_overlay=TextOverlay(visible=False, color=(0.5, 0.5, 0.5, 1.0), font_size=10, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), overlays=Overlays(interaction_box=InteractionBox(points=None, show=False, show_handle=False, show_vertices=False, sele

In [13]:
#len(fls_all)

In [3]:
def new_segmentation_subtr(fl =r'\\192.168.0.100\bbfish100\DCBBL1_4week_6_2_2023\H1_MER_set1\Conv_zscan__030.zarr',
                     psf_file = '\\\\192.168.0.100\\bbfish100\\DCBBL1_4week_6_2_2023\\MERFISH_Analysis\\psf_750_Scope3_final.npy',
                           subtr_bk=True,
                     p1=-500,p99=1500,mean_dapi = None,sdapi = 100,
                    save_folder = r'\\192.168.0.100\bbfish100\DCBBL1_4week_6_2_2023\MERFISH_Analysis',redo=False,plt_val=False,st = 5):

    segm_folder = save_folder+os.sep+'Segmentation'
    if not os.path.exists(segm_folder): os.makedirs(segm_folder)
    fl_dapi = fl
    save_fl  = segm_folder+os.sep+os.path.basename(fl_dapi).split('.')[0]+'--'+os.path.basename(os.path.dirname(fl_dapi))+'--dapi_segm.npz'
    if redo or (not os.path.exists(save_fl)):
    
        im = read_im(fl)
        im_dapi = np.array(im[-1],dtype=np.float32)
        imd = im_dapi
        if subtr_bk:
            im_bk = np.array(im[-2],dtype=np.float32)
            ifr = len(im_dapi)//2
            fr = im_bk[ifr]/im_dapi[ifr]
            fr_ = np.max(fr)
            im_dapi = im_dapi-im_bk/fr_
        
        if psf_file is not None:
            psf = np.load(psf_file)
            imd = full_deconv(im_dapi,s_=500,pad=100,psf=psf,parameters={'method': 'wiener', 'beta': 0.01},gpu=True,force=False)
        
        im_dapi_ = norm_slice(imd,s=sdapi)


        img = np.array(np.clip((im_dapi_[::3,::4,::4]-p1)/(p99-p1),0,1),dtype=np.float32)
        imd_ = imd[::3,::4,::4]

        from cellpose import models, io,utils
        from scipy import ndimage
        model = models.Cellpose(gpu=True, model_type='cyto')
        masks, flows, styles, diams = model.eval(img,z_axis=0, diameter=20, channels=[0,0],
                                                 flow_threshold=-10,cellprob_threshold=-10,normalize=False,
                                                 do_3D=False,stitch_threshold=0.,
                                                 progress=True)

        for ifr in range(len(masks)):
            means = nd.mean(np.clip(imd_[ifr],0,np.max(imd_[ifr])),masks[ifr],np.unique(masks[ifr]))
            
            if mean_dapi is None: mean_dapi = (st*means[0]+np.median(means[1:]))/(st+1)
            bad_cells = np.where(means<mean_dapi)[0]
            masks[ifr] = replace_mat(masks[ifr],bad_cells,0)
        if plt_val:
            import napari
            v = napari.view_image(img)
            v.add_labels(masks)
        shape = np.array(im[-1].shape)
        np.savez_compressed(save_fl,segm = masks,shape = shape)
    return save_fl

In [41]:
#save_folder =r'\\192.168.0.100\bbfish100\DCBBL1_1year1wkNJ_ASO_SAL_4_18_2023\MERFISH_Analysis'
save_folder = master_data_folder = r'\\192.168.0.21\bbfishdc21\HttHuman_Snonia__06_12_2023'
if True:
    fl = r'\\192.168.0.21\bbfishdc21\HttHuman_Snonia__06_12_2023\H0__set1\Conv_zscan__30.zarr'
    new_segmentation_subtr(fl =fl,subtr_bk=True,
                             psf_file = r'C:\Scripts\NMERFISH\psfs\psf_750_Scope3_final.npy',
                             p1=-500,p99=5000,mean_dapi = None,sdapi = 100,
                            save_folder = save_folder,redo=True,plt_val=True,st=1)

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:04<00:00,  7.99it/s]


2023-07-06 07:45:48,093 [INFO] ** TORCH GPU version installed and working. **
2023-07-06 07:45:48,094 [INFO] >>>> using GPU
2023-07-06 07:45:48,155 [WARNING] cellprob_threshold and dist_threshold are being deprecated in a future release, use mask_threshold instead
2023-07-06 07:45:48,156 [INFO] ~~~ FINDING MASKS ~~~
2023-07-06 07:45:48,157 [INFO] multi-stack tiff read in as having 13 planes 1 channels
2023-07-06 07:46:17,464 [INFO] 100%|##########| 13/13 [00:29<00:00,  2.25s/it]
2023-07-06 07:46:17,464 [INFO] network run in 29.28s
2023-07-06 07:46:19,300 [INFO] masks created in 1.84s
2023-07-06 07:46:19,859 [INFO] >>>> TOTAL TIME 31.70 sec


In [4]:
fls_all = glob.glob(r'\\192.168.0.21\bbfishdc21\HttHuman_Snonia__06_12_2023\H0__set*\*.zarr')

In [5]:
fls_all = glob.glob(r'\\192.168.0.21\bbfishdc21\HttHuman_Snonia__06_12_2023\H0__set*\*.zarr')
save_folder =r'\\192.168.0.21\bbfishdc21\HttHuman_Snonia__06_12_2023'
for fl in tqdm(fls_all[::-1]):
    try:
        new_segmentation_subtr(fl =fl,subtr_bk=True,
                             psf_file = r'C:\Scripts\NMERFISH\psfs\psf_750_Scope3_final.npy',
                             p1=-500,p99=5000,mean_dapi = None,sdapi = 100,
                            save_folder = save_folder,redo=False,plt_val=False,st=1)
    except:
        print("Failed",fl)

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:07<00:00,  4.66it/s]


2023-07-06 07:53:20,302 [INFO] ** TORCH GPU version installed and working. **
2023-07-06 07:53:20,303 [INFO] >>>> using GPU
2023-07-06 07:53:20,380 [WARNING] cellprob_threshold and dist_threshold are being deprecated in a future release, use mask_threshold instead
2023-07-06 07:53:20,381 [INFO] ~~~ FINDING MASKS ~~~
2023-07-06 07:53:20,381 [INFO] multi-stack tiff read in as having 13 planes 1 channels
2023-07-06 07:53:20,404 [INFO] 
2023-07-06 07:53:20,405 [INFO] 0%|          | 0/13 [00:00<?, ?it/s]
2023-07-06 07:53:20,405 [INFO] 
2023-07-06 07:53:24,850 [INFO] 
2023-07-06 07:53:24,850 [INFO] 8%|7         | 1/13 [00:04<00:53,  4.44s/it]
2023-07-06 07:53:24,850 [INFO] 
2023-07-06 07:53:27,126 [INFO] 
2023-07-06 07:53:27,127 [INFO] 15%|#5        | 2/13 [00:06<00:34,  3.17s/it]
2023-07-06 07:53:27,128 [INFO] 
2023-07-06 07:53:29,395 [INFO] 
2023-07-06 07:53:29,395 [INFO] 23%|##3       | 3/13 [00:08<00:27,  2.76s/it]
2023-07-06 07:53:29,395 [INFO] 
2023-07-06 07:53:31,621 [INFO] 
2023-07-0

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:04<00:00,  8.07it/s]


2023-07-06 07:54:12,230 [INFO] ** TORCH GPU version installed and working. **
2023-07-06 07:54:12,230 [INFO] >>>> using GPU
2023-07-06 07:54:12,291 [WARNING] cellprob_threshold and dist_threshold are being deprecated in a future release, use mask_threshold instead
2023-07-06 07:54:12,291 [INFO] ~~~ FINDING MASKS ~~~
2023-07-06 07:54:12,291 [INFO] multi-stack tiff read in as having 13 planes 1 channels
2023-07-06 07:54:12,319 [INFO] 
2023-07-06 07:54:12,320 [INFO] 0%|          | 0/13 [00:00<?, ?it/s]
2023-07-06 07:54:12,320 [INFO] 
2023-07-06 07:54:14,667 [INFO] 
2023-07-06 07:54:14,667 [INFO] 8%|7         | 1/13 [00:02<00:28,  2.35s/it]
2023-07-06 07:54:14,667 [INFO] 
2023-07-06 07:54:16,955 [INFO] 
2023-07-06 07:54:16,955 [INFO] 15%|#5        | 2/13 [00:04<00:25,  2.31s/it]
2023-07-06 07:54:16,955 [INFO] 
2023-07-06 07:54:19,185 [INFO] 
2023-07-06 07:54:19,185 [INFO] 23%|##3       | 3/13 [00:06<00:22,  2.27s/it]
2023-07-06 07:54:19,185 [INFO] 
2023-07-06 07:54:21,419 [INFO] 
2023-07-0

C:\Users\BintuLab\anaconda3\envs\cellpose\lib\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\BintuLab\anaconda3\envs\cellpose\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:04<00:00,  8.00it/s]


...

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:04<00:00,  7.92it/s]


2023-07-06 12:33:23,673 [INFO] ** TORCH GPU version installed and working. **
2023-07-06 12:33:23,681 [INFO] >>>> using GPU
2023-07-06 12:33:23,742 [WARNING] cellprob_threshold and dist_threshold are being deprecated in a future release, use mask_threshold instead
2023-07-06 12:33:23,743 [INFO] ~~~ FINDING MASKS ~~~
2023-07-06 12:33:23,744 [INFO] multi-stack tiff read in as having 13 planes 1 channels
2023-07-06 12:33:23,767 [INFO] 
2023-07-06 12:33:23,768 [INFO] 0%|          | 0/13 [00:00<?, ?it/s]
2023-07-06 12:33:23,768 [INFO] 
2023-07-06 12:33:26,203 [INFO] 
2023-07-06 12:33:26,203 [INFO] 8%|7         | 1/13 [00:02<00:29,  2.43s/it]
2023-07-06 12:33:26,203 [INFO] 
2023-07-06 12:33:28,459 [INFO] 
2023-07-06 12:33:28,459 [INFO] 15%|#5        | 2/13 [00:04<00:25,  2.33s/it]
2023-07-06 12:33:28,459 [INFO] 
2023-07-06 12:33:30,705 [INFO] 
2023-07-06 12:33:30,705 [INFO] 23%|##3       | 3/13 [00:06<00:22,  2.29s/it]
2023-07-06 12:33:30,713 [INFO] 
2023-07-06 12:33:32,967 [INFO] 
2023-07-0

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:04<00:00,  8.14it/s]


2023-07-06 12:34:16,928 [INFO] ** TORCH GPU version installed and working. **
2023-07-06 12:34:16,928 [INFO] >>>> using GPU
2023-07-06 12:34:16,984 [WARNING] cellprob_threshold and dist_threshold are being deprecated in a future release, use mask_threshold instead
2023-07-06 12:34:16,984 [INFO] ~~~ FINDING MASKS ~~~
2023-07-06 12:34:16,992 [INFO] multi-stack tiff read in as having 13 planes 1 channels
2023-07-06 12:34:17,020 [INFO] 
2023-07-06 12:34:17,020 [INFO] 0%|          | 0/13 [00:00<?, ?it/s]
2023-07-06 12:34:17,021 [INFO] 
2023-07-06 12:34:19,398 [INFO] 
2023-07-06 12:34:19,399 [INFO] 8%|7         | 1/13 [00:02<00:28,  2.38s/it]
2023-07-06 12:34:19,400 [INFO] 
2023-07-06 12:34:21,673 [INFO] 
2023-07-06 12:34:21,673 [INFO] 15%|#5        | 2/13 [00:04<00:25,  2.32s/it]
2023-07-06 12:34:21,673 [INFO] 
2023-07-06 12:34:23,950 [INFO] 
2023-07-06 12:34:23,950 [INFO] 23%|##3       | 3/13 [00:06<00:22,  2.30s/it]
2023-07-06 12:34:23,950 [INFO] 
2023-07-06 12:34:26,210 [INFO] 
2023-07-0

100%|██████████████████████████████████████████████████████████████████████████████| 317/317 [4:41:55<00:00, 53.36s/it]


In [9]:
from cellpose import models, io,utils
from scipy import ndimage
model = models.Cellpose(gpu=True, model_type='nuclei')

p1,p99=0,np.percentile(imd[::3,::4,::4],97)#8000
img = np.array(np.clip((imd[::3,::4,::4]-p1)/(p99-p1),0,1),dtype=np.float32)
masks, flows, styles, diams = model.eval(img,z_axis=0, diameter=15, channels=[0,0],
                                         flow_threshold=-10,cellprob_threshold=-10,normalize=False,
                                         do_3D=False,stitch_threshold=0.5,
                                         progress=True)

In [40]:
v = napari.view_image(img)
v.add_labels(masks)

<Labels layer 'masks' at 0x1a9ebcddc70>

In [14]:
np.percentile(im_dapi_[::3,::4,::4],99)

2423.224697265623

In [10]:
napari.view_image(im_dapi_)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 1499.5, 1499.5), zoom=0.17575000000000002, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(18.0, 1.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=3, ndisplay=2, last_used=0, range=((0.0, 37.0, 1.0), (0.0, 3000.0, 1.0), (0.0, 3000.0, 1.0)), current_step=(18, 1500, 1500), order=(0, 1, 2), axis_labels=('0', '1', '2')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'im_dapi_' at 0x1a6beb58bb0>], scale_bar=ScaleBar(visible=False, colored=False, color=array([1., 0., 1., 1.], dtype=float32), ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, box=False, box_color=array([0. , 0. , 0. , 0.6], dtype=float32), unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ], dtype=float32), font_size=10.0, position=<TextOverlayPos

In [4]:
psf = np.load('\\\\192.168.0.100\\bbfish100\\DCBBL1_4week_6_2_2023\\MERFISH_Analysis\\psf_750_Scope3_final.npy')

In [39]:
imd = full_deconv(im_dapi,s_=500,pad=100,psf=psf,parameters={'method': 'wiener', 'beta': 0.01},gpu=True,force=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 36/36 [00:04<00:00,  8.07it/s]


In [40]:
from cellpose import models, io,utils
from scipy import ndimage
model = models.Cellpose(gpu=True, model_type='nuclei')


In [46]:
np.percentile(imd,97)

12677.7783203125

In [49]:
v = napari.view_image(img)
v.add_labels(masks)

<Labels layer 'masks' at 0x1e619b98790>

In [111]:
from cellpose import models, io,utils
from scipy import ndimage
model = models.Cellpose(gpu=True, model_type='nuclei')
masksA = []
for img in tqdm(imd[::3,::4,::4]):
    p1 = 0#np.percentile(img,1)
    #if p99 is None:
    p99 = 23000#np.percentile(img,99.)
    img = np.array(np.clip((img-p1)/(p99-p1),0,1),dtype=np.float32)
    masks, flows, styles, diams = model.eval(img, diameter=15, channels=[0,0],
                                         flow_threshold=1,cellprob_threshold=-10,normalize=False)
    masksA.append(masks)

100%|██████████████████████████████████████████████████████████████████████████████████| 13/13 [01:34<00:00,  7.23s/it]


In [112]:
masksA = np.array(masksA)

In [113]:
v = napari.view_image(imd[::3,::4,::4],contrast_limits=[p1,p99])
v.add_labels(masksA)

<Labels layer 'masksA' at 0x240c553ac70>

In [28]:
import napari
v = napari.view_image(imd)
v.add_image(im_dapi)

<Image layer 'im_dapi' at 0x23e3486bfd0>

In [17]:
napari.view_image(im_dapi);

In [ ]:
psf=None,
    plt_val=False,
    parameters={'method': 'wiener', 'beta': 0.001, 'niter': 50},
    gpu=False,
    force=False,
    pad=None,

In [22]:
def full_deconv(im_,s_=500,pad=100,psf=None,parameters={'method': 'wiener', 'beta': 0.001, 'niter': 50},gpu=True,force=False):
    im0=np.zeros_like(im_)
    sx,sy = im_.shape[1:]
    ixys = []
    for ix in np.arange(0,sx,s_):
        for iy in np.arange(0,sy,s_):
            ixys.append([ix,iy])
    
    for ix,iy in tqdm(ixys):#ixys:#tqdm(ixys):
        imsm = im_[:,ix:ix+pad+s_,iy:iy+pad+s_]
        imt = apply_deconv(imsm,psf=psf,parameters=parameters,gpu=gpu,plt_val=False,force=force)
        start_x = ix+pad//2 if ix>0 else 0
        end_x = ix+pad//2+s_
        start_y = iy+pad//2 if iy>0 else 0
        end_y = iy+pad//2+s_
        #print(start_x,end_x,start_y,end_y)
        im0[:,start_x:end_x,start_y:end_y] = imt[:,(start_x-ix):(end_x-ix),(start_y-iy):(end_y-iy)]
    return im0

In [14]:
#[ln for ln in open(r'\\192.168.0.100\bbfish100\DCBBL1_4week_6_2_2023\H1_MER_set2\Conv_zscan__010.power','r')]

In [15]:
#np.array(im[9])

In [ ]:
"""Using cellpose with nuclei mode"""
from cellpose import models, io,utils
from scipy import ndimage
model = models.Cellpose(gpu=use_gpu, model_type=model)
#decided that resampling to the 4-2-2 will make it faster
#im_dapi_3d = im_dapi[::rescz,::resc,::resc].astype(np.float32)
chan = [0,0]
masks_all = []
flows_all = []
from tqdm import tqdm
for im in tqdm(im_dapi):
    im_ = np.array(im,dtype=np.float32)
    img = (cv2.blur(im_,(2,2))-cv2.blur(im_,(150,150)))[::resc,::resc]
    p1 = np.percentile(img,1)
    if p99 is None:
        p99 = np.percentile(img,99.9)
    img = np.array(np.clip((img-p1)/(p99-p1),0,1),dtype=np.float32)
    masks, flows, styles, diams = model.eval(img, diameter=sz_cell, channels=chan,
                                         flow_threshold=0.85,cellprob_threshold=-2,min_size=sz_min_2d,normalize=False)

### Save psf

In [4]:
psfs = []

In [ ]:
master_folder = r'\\192.168.0.100\bbfish100\DCBBL1_4week_6_2_2023\P2_Sox11_Sox2_Dcx__set2'

for ifov in tqdm([11,12,13,14,15,16,17,18]):
    im = read_im(master_folder+r'\Conv_zscan__'+str(ifov).zfill(3)+'.zarr')
    im_ = np.array(im[0][1:,500:2500,500:2500],dtype=np.float32)
    psf = get_psf(im_,th=1000,th_cor = 0.75,delta=3,delta_fit = 7,sxyzP = [15,60,60])
    psfs.append(psf)
#napari.view_image(im)
psff = np.mean([psf for psf in psfs if psf is not None],axis=0)
psff_ = np.array([p-np.median(p) for p in psff])
from scipy.ndimage import median_filter
psff_med = median_filter(psff_, size=15)
psfff = (psff_-psff_med)[5:-5,5:-5,5:-5][:-1,:-1,:-1]
psfff[psfff<0]=0
psfff = psfff/np.max(psfff)
np.save('psf_750_Scope3_final.npy',psfff)

In [16]:
if False:
    imd = apply_deconv(im_[:,750:-750,750:-750],
        psf=psf,
        plt_val=True,
        parameters={'method': 'wiener', 'beta': 0.0001, 'niter': 50},
        gpu=False,
        force=True,
        pad=None)

In [22]:
import shutil
save_folder = os.path.dirname(master_folder)+os.sep+'MERFISH_Analysis'
if not os.path.exists(save_folder): os.makedirs(save_folder)
shutil.copy2(os.path.abspath('psf_750_Scope3_final.npy'),save_folder+os.sep+'psf_750_Scope3_final.npy')

'\\\\192.168.0.100\\bbfish100\\DCBBL1_4week_6_2_2023\\MERFISH_Analysis\\psf_750_Scope3_final.npy'

### Save flat field

In [24]:
#np.load(r'\\192.168.0.100\bbfish100\DCBBL1_1year1wkNJ_ASO_SAL_4_18_2023\MERFISH_Analysis\med_col0.npz')['im'].shape

(300, 300)

In [27]:
data_fld = r'\\192.168.0.100\bbfish100\DCBBL1_4week_6_2_2023\P2_Sox11_Sox2_Dcx__set1'
save_folder = os.path.dirname(master_folder)+os.sep+'MERFISH_Analysis'
if not os.path.exists(save_folder): os.makedirs(save_folder)

In [ ]:
compute_flat_field_raw(data_fld=data_fld,
    save_folder=save_folder,
    ncols=4)

 94%|███████████████████████████████████████████████████████████████████████████▏    | 656/698 [04:16<00:19,  2.20it/s]